In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
df=pd.read_excel('sample_data/Balanced Training data.xlsx')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc

In [ ]:
df.head()

,HHKEY,ZIP_CODE,REC,FRE,MON,CC_CARD,AVRG,PC_CALC20,PSWEATERS,PKNIT_TOPS,PKNIT_DRES,PBLOUSES,PJACKETS,PCAR_PNTS,PCAS_PNTS,PSHIRTS,PDRESSES,PSUITS,POUTERWEAR,PJEWELRY,PFASHION,PLEGWEAR,PCOLLSPND,AMSPEND,PSSPEND,CCSPEND,AXSPEND,TMONSPEND,OMONSPEND,SMONSPEND,PREVPD,GMP,PROMOS,DAYS,FREDAYS,MARKDOWN,CLASSES,COUPONS,STYLES,STORES,...,Unnamed: 198,Unnamed: 199,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206,Unnamed: 207,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221,Unnamed: 222,Unnamed: 223,Unnamed: 224,Unnamed: 225,Unnamed: 226,Unnamed: 227,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237
0,9960000000000,1001,208,2,368.46,0,184.23,11,0.18,0.0,0.00,0.30,0.00,0.25,0.00,0.19,0.0,0.0,0.0,0.000000,0.02,0.03,0.29,0.0,0.0,368.46,0.0,0.00,0.0,0.00,0.00,0.60,17,666,333.00,0.08,9,1,11,1,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,19.195312,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.808142,2.197225,1.000000,2.397895,0.0,4.709530,3.459781,0.00000,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0
1,9960000000000,1056,327,2,77.00,0,38.50,11,1.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.000000,0.00,0.00,0.00,0.0,0.0,77.00,0.0,0.00,0.0,0.00,39.00,0.62,10,343,171.50,0.11,1,0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.774964,0.0,0.000000,0.0,0.000000,6.244998,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.144583,0.000000,0.000000,0.693147,0.0,4.228293,4.605170,0.00000,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0
2,9960000000000,1118,66,8,846.06,1,105.75,11,0.38,0.0,0.05,0.06,0.20,0.17,0.00,0.05,0.0,0.0,0.0,0.005307,0.03,0.01,0.00,0.0,0.0,846.06,0.0,104.94,0.0,373.87,166.25,0.43,24,701,87.62,0.29,15,3,35,1,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,29.087111,0.0,10.244023,0.0,19.335718,12.893797,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,4.473009,2.708050,1.732051,3.555348,0.0,3.294354,3.147165,2.44949,0.0,1.0,1.0,10.244023,16.399085,1.0,1.0
3,9960000000000,1107,49,1,87.44,0,87.44,11,0.20,0.2,0.00,0.00,0.00,0.00,0.41,0.00,0.0,0.0,0.0,0.170000,0.00,0.00,0.00,0.0,0.0,87.44,0.0,87.44,0.0,87.44,0.00,0.22,0,49,49.00,0.42,4,0,8,1,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9.350936,0.0,9.350936,0.0,9.350936,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.891820,1.386294,0.000000,2.079442,0.0,3.198673,3.350606,0.00000,0.0,0.0,1.0,9.350936,0.000000,0.0,1.0
4,9960000000000,1118,145,1,782.08,1,782.08,11,0.12,0.0,0.00,0.00,0.57,0.12,0.00,0.17,0.0,0.0,0.0,0.000000,0.00,0.00,0.00,0.0,0.0,782.08,0.0,0.00,0.0,782.08,0.00,0.66,15,145,145.00,0.10,4,1,13,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.965693,0.0,0.000000,0.0,27.965693,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,4.976734,1.386294,1.000000,2.564949,0.0,3.590439,3.676554,0.00000,0.0,0.0,1.0,0.000000,27.965693,1.0,0.0


In [ ]:
df.shape

(7673, 238)

In [ ]:
vars = ['CLUSTYPE', 'WEB', 'AXSPEND', 'AMSPEND','HHKEY',
'ZIP_CODE',
'REC',
'FRE',
'MON',
'CC_CARD',
'AVRG',
'PC_CALC20',
'PSWEATERS',
'PKNIT_TOPS',
'PKNIT_DRES',
'PBLOUSES',
'PJACKETS',
'PCAR_PNTS',
'PCAS_PNTS',
'PSHIRTS',
'PDRESSES',
'PSUITS',
'POUTERWEAR',
'PJEWELRY',
'PFASHION',
'PLEGWEAR',
'sqrt sweaters',
'sqrt knit tops',
'sqrt knit dresses',
'sqrt blouses',
'sqrt jackets',
'sqrt career pants',
'sqrt casual pants',
'sqrt shirts',
'sqrt dresses',
'sqrt suits',
'sqrt outerwear',
'sqrt jewelry',
'sqrt fashion',
'sqrt legwear',
'sqrt collectibles',
'flag sweaters',
'flag knit tops',
'flag knit dresses',
'flag blouses',
'flag jackets',
'flag career pants',
'flag casual pants',
'flag shirts',
'flag dresses',
'flag suits',
'flag outerwear',
'flag jewelry',
'flag fashion',
'flag legwear',
'flag collectibles',
'sqrt spending AM']

In [ ]:
y_cols = 'RESP'
x_cols = vars
# x_cols.remove(y_cols)

In [ ]:
dfdtype = pd.DataFrame(df.dtypes)
flag_cols = list(dfdtype[dfdtype.iloc[:,0] == 'object'].index)

In [ ]:
df = df.replace(['Y', 'N'], [1, 0])
df = df.replace(['T', 'F'], [1, 0])

In [ ]:
df['RESP'].value_counts()

0    3841
1    3832
Name: RESP, dtype: int64

In [ ]:
zerodf = df[df[y_cols]== 0].sample(3841)
onedf = df[df[y_cols]== 1].sample(3832)

newdf = pd.concat([zerodf, onedf], axis=0)
newdf[y_cols].value_counts()

0    3841
1    3832
Name: RESP, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import neighbors
newdf = newdf.dropna()

In [ ]:
# Helper functions to calculate the performance of our models.
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=2)
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(str(title.split('\n')[0])+'.png')
    plt.show()
    
def overall_error_rate(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    fn = cnf_matrix[1,0]
    fp = cnf_matrix[0,1]
    tn = cnf_matrix[0,0]
    tp = cnf_matrix[1,1]
    n = len(y_test)
    return (fn+fp)/n

def sensitivity(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    tap = pd.DataFrame(y_test).iloc[:,0].value_counts()[1]
    tp = cnf_matrix[1,1]
    return tp/tap

def false_pos_rate(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    fp = cnf_matrix[0,1]
    tan = pd.DataFrame(y_test).iloc[:,0].value_counts()[0]
    return fp/tan

def specificity(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    tn = cnf_matrix[0,0]
    tan = pd.DataFrame(y_test).iloc[:,0].value_counts()[0]
    return tn/tan

def false_neg_rate(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    fn = cnf_matrix[1,0]
    tap = pd.DataFrame(y_test).iloc[:,0].value_counts()[1]
    return fn/tap

def prop_true_pos(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    try:
        tpp = pd.DataFrame(y_pred).iloc[:,0].value_counts()[1]
    except:
        return 0
    tp = cnf_matrix[1,1]
    return tp/tpp

def prop_true_neg(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    try:
        tn = cnf_matrix[0,0]
        tpn = pd.DataFrame(y_pred).iloc[:,0].value_counts()[0]
    except:
        return 0
    return tn/tpn

def recall(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    try:
        tp = cnf_matrix[1,1]
        fn = cnf_matrix[1,0]
        tpn = pd.DataFrame(y_pred).iloc[:,0].value_counts()[0]
    except:
        return 0
    return tp/(fn+tp)

def precision(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    try:
        tp = cnf_matrix[1,1]
        fp = cnf_matrix[0,1]
        tpn = pd.DataFrame(y_pred).iloc[:,0].value_counts()[0]
    except:
        return 0
    return tp/(fp+tp)

def npv(y_pred, y_test):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    try:
        fn = cnf_matrix[1,0]
        fn = cnf_matrix[1,0]
        tn = cnf_matrix[0,0]
        tpn = pd.DataFrame(y_pred).iloc[:,0].value_counts()[0]
    except:
        return 0
    return tn/(tn+fn)

def f1score(y_pred, y_test):
    prec = precision(y_pred, y_test)
    rec = recall(y_pred, y_test)
    f1 = 2 * ((prec * rec)/(prec + rec))
    return f1

def get_descriptive_data(y_pred, y_test):
    print("Accuracy: %f%%" %(round(accuracy_score(y_test, y_pred)*100,2)))
    print("Overall Error Rate: %f%%" %(round(overall_error_rate(y_pred, y_test)*100,2)))
    print('False Positive Rate: %f%%' %(round(false_pos_rate(y_pred, y_test)*100,2)))
    print('False Negative Rate: %f%%' %(round(false_neg_rate(y_pred, y_test)*100,2)))
    print('Specificity: %f%%' %(round(specificity(y_pred, y_test)*100,2)))
    print("Sensitivity: %f%%" %(round(sensitivity(y_pred, y_test)*100,2)))
    print('Proportion True Positive: %f%%' %(round(prop_true_pos(y_pred, y_test)*100,2)))
    print('Proportion True Negative: %f%%' %(round(prop_true_neg(y_pred, y_test)*100,2)))
    print("recall: %f%%" %(round(recall(y_pred, y_test)*100,2)))
    print("precision: %f%%" %(round(precision(y_pred, y_test)*100,2)))
    print("FDR: %f%%" %(100-round(precision(y_pred, y_test)*100,2)))
    print("NPV: %f%%" %(round(precision(y_pred, y_test)*100,2)))
    print("FOR: %f%%" %(100-round(npv(y_pred, y_test)*100,2)))
    print("F1SCORE: %f%%" %(100-round(f1score(y_pred, y_test)*100,2)))

In [ ]:
y_cols = 'RESP'
x_cols = vars
# x_cols.remove(y_cols)

In [ ]:
dfdtype = pd.DataFrame(df.dtypes)
flag_cols = list(dfdtype[dfdtype.iloc[:,0] == 'object'].index)

In [ ]:
df = df.replace(['Y', 'N'], [1, 0])
df = df.replace(['T', 'F'], [1, 0])

In [ ]:
df['RESP'].value_counts()

0    3841
1    3832
Name: RESP, dtype: int64

In [ ]:
df[y_cols].value_counts()

0    3841
1    3832
Name: RESP, dtype: int64

In [ ]:
newdf[y_cols].value_counts()

0    3615
Name: RESP, dtype: int64